In [1]:
import pyspark 
from pyspark.sql import SparkSession


### Question 1

In [2]:
pyspark.__version__

'3.3.2'

### Question 2

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("fhv_tripdata_2021-06.csv")

In [5]:
df.head(5)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 16, 16), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 21, 14), PULocationID=32, DOLocationID=254, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 27, 1), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 42, 11), PULocationID=240, DOLocationID=127, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 46, 8), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 53, 45), PULocationID=127, DOLocationID=235, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02510', pickup_datetime=datetime.d

In [7]:
### Repartition it to 12 partitions and save it to parquet

df.repartition(12).write.parquet("fhv_tripdata_2021-06.parquet")

In [13]:
### Average size of the parquet files in MB

import os
import glob

parquet_files = glob.glob("fhv_tripdata_2021-06.parquet/*.parquet")
parquet_files_size = [os.path.getsize(file) for file in parquet_files]
a = sum(parquet_files_size) / len(parquet_files_size) / 1024 / 1024

print('Answer:', a, 'MB')

Answer: 22.421844879786175 MB


### Question 3

In [15]:
df_fhv = spark.read.parquet('fhv_tripdata_2021-06.parquet/*')

In [18]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02875|2021-06-03 16:28:28|2021-06-03 17:01:11|          15|         145|      N|                B02875|
|              B02887|2021-06-01 09:41:03|2021-06-01 10:02:55|          95|         170|      N|                B02887|
|              B02882|2021-06-03 16:07:59|2021-06-03 16:43:18|         161|          87|      N|                B02882|
|              B02889|2021-06-03 23:33:49|2021-06-03 23:39:09|          21|          21|      N|                B02889|
|              B02867|2021-06-02 18:20:05|2021-06-02 21:33:56|          92|         265|      N|                B02867|
|              B02887|2021-06-01 16:37:2

In [16]:
df_fhv.registerTempTable('fhv')

c:\Users\User\anaconda3\envs\data_engineering_camp\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
df_fhv_june_15 = spark.sql("""
SELECT 
    count(*)
FROM fhv 
WHERE pickup_datetime BETWEEN '2021-06-15 00:00:00' AND '2021-06-15 23:59:59'

""")

In [22]:
df_fhv_june_15.show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



### Question 4

In [46]:
df_fhv_longest_trip = spark.sql ("""
SELECT CAST(pickup_datetime AS DATE) AS FECHA,
    MAX(unix_timestamp(dropoff_datetime)- unix_timestamp(pickup_datetime))/3600
FROM fhv
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 1
""")

In [47]:
df_fhv_longest_trip.show()

+----------+----------------------------------------------------------------------------------------------------------------------------+
|     FECHA|(max((unix_timestamp(dropoff_datetime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(pickup_datetime, yyyy-MM-dd HH:mm:ss))) / 3600)|
+----------+----------------------------------------------------------------------------------------------------------------------------+
|2021-06-25|                                                                                                            66.8788888888889|
+----------+----------------------------------------------------------------------------------------------------------------------------+



### Question 5

Answer: 4040

### Question 6

In [48]:
df_taxi_zone_lookup = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("taxi_zone_lookup.csv")


In [49]:
df_taxi_zone_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [50]:
df_taxi_zone_lookup.registerTempTable('taxi_zone_lookup')

c:\Users\User\anaconda3\envs\data_engineering_camp\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [53]:
df_result = spark.sql(""" 
SELECT COUNT(pul_zone.Zone), pul_zone.Zone
FROM fhv 
JOIN taxi_zone_lookup pul_zone ON fhv.PULocationID = pul_zone.LocationID
JOIN taxi_zone_lookup dol_zone ON fhv.DOLocationID = dol_zone.LocationID
GROUP BY pul_zone.Zone
ORDER BY 1 DESC 
""")

In [54]:
df_result.show()

+-----------+--------------------+
|count(Zone)|                Zone|
+-----------+--------------------+
|     231279| Crown Heights North|
|     221244|        East Village|
|     188867|         JFK Airport|
|     187929|      Bushwick South|
|     186780|       East New York|
|     164344|TriBeCa/Civic Center|
|     161596|   LaGuardia Airport|
|     158937|            Union Sq|
|     154698|        West Village|
|     152493|             Astoria|
|     151020|     Lower East Side|
|     147673|        East Chelsea|
|     146402|Central Harlem North|
|     143683|Williamsburg (Nor...|
|     143594|          Park Slope|
|     141427|  Stuyvesant Heights|
|     139611|        Clinton East|
|     139431|West Chelsea/Huds...|
|     138428|             Bedford|
|     137879|         Murray Hill|
+-----------+--------------------+
only showing top 20 rows

